In [52]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee as mc
from scipy import stats

%matplotlib inline

In [53]:
def createMockCluster(rin,rout,nmean,k,smax,m,vmean,threeD=False):
    ntot = np.random.poisson(lam=nmean)
    
    nu0 = ntot/(4*np.pi*(np.log(rout/rin))) if k==3. else ntot*(3.-k)/(4*np.pi*(rout**(3.-k)-rin**(3.-k)))
    #rho = 4*np.pi*nu0*np.random.uniform(low=rin,high=rout,size=ntot)**(2.-k)
    rho = np.power(np.random.uniform(4*np.pi*nu0*rin**(2.-k),4*np.pi*nu0*rout**(2.-k),size=ntot)/(4*np.pi*nu0),1./(2-k))
    phi = np.random.uniform(0.,2*np.pi,ntot)
    theta = np.arccos(np.random.uniform(-1,1,ntot))
    
    x = rho*np.sin(theta)*np.cos(phi)
    y = rho*np.sin(theta)*np.sin(phi)
    
    vobs = np.array([np.random.normal(loc=vmean,scale=(1/np.sqrt(3))*smax/(i**m)) for i in rho])
    
    if threeD:
        z = rho*np.cos(theta)
        return pd.DataFrame(np.hstack((x.reshape((ntot,1)),y.reshape((ntot,1)),z.reshape((ntot,1)),\
                                      vobs.reshape((ntot,1)),rho.reshape((ntot,1)))),columns=['x','y','z','vobs','rho'])
    else:
        r = np.sqrt(np.power(x,2.)+np.power(y,2.))
        return pd.DataFrame(np.hstack((r.reshape((ntot,1)),vobs.reshape((ntot,1)))),columns=['r','vobs'])

In [54]:
#load data

data=pd.read_csv('continuousMock0.csv')

def defineAnnuli(r,nAnn):
    return np.append(0.,np.sort(r)[np.linspace(0,len(r)-1,nAnn).astype(int)][1:])

nAnn = 10
annuli = defineAnnuli(data['r'],nAnn)

In [55]:
def labelGalaxies(data,annuli):
    #adds a column to data indicating the galaxy's membership in an annulus
    bla = np.zeros(len(data['r']))
    for i in range(len(annuli)-1):
        bla += i*np.array(np.logical_and(data['r']>annuli[i],data['r']<=annuli[i+1]).astype(int))
    return bla

def galInAnn(data,annuli):
    #calculates the number of galaxies in all the annuli
    return np.array([(data['annuli']==i).sum() for i in range(len(annuli)-1)])

def sigInAnn(data,annuli):
    return [np.std(data['vobs'][data['annuli']==i],ddof=1) for i in range(len(annuli)-1)]

def getProfiles(annuli,rin,rout,nmean,k,smax,m,vmean):
    mock = createMockCluster(rin,rout,nmean,k,smax,m,vmean)
    mock['annuli'] = labelGalaxies(mock,annuli)
    return galInAnn(mock,annuli),sigInAnn(mock,annuli)

In [56]:
data['annuli'] = labelGalaxies(data,annuli)

nObs = galInAnn(data,annuli)
sObs = sigInAnn(data,annuli)

print nObs,sObs

[112 111 112 111 112 111 112 111 112] [540.6459164824956, 535.2118446805213, 479.94459127400813, 478.113651156466, 505.3241440991127, 495.1272316797748, 450.3661464577093, 348.32470090040425, 383.5558324031583]


In [57]:
#all log likelihoods

#make a table of log(n!)-s for n=1,...
#factorialTable = np.cumsum(np.append(0,[np.log(i) for i in range(1,len(data['r']))]))

def logPriorK(k):
    #starting gaussian
    #nu0 = 3. #mean prior density index
    #sigma = 1.
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorM(m):
    #starting gaussian
    #nu0 = -.2 #mean prior sigma index
    #sigma = 0.1
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorSigma0(sigma0):
    #starting gaussian
    #nu0 = 1000 #mean prior sigma index
    #sigma = 500
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorNmean(nmean):
    return 0.0

def logPriorNtot(ntot):
    return 0.0

def logLikelihoodNu(prediction,data):
    return np.sum(data*np.log(prediction)-prediction)

def logLikelihoodSigma(vObs,vPred,nPred):
    # distribution of variances is chisq, as shown, for example, at
    # https://onlinecourses.science.psu.edu/stat414/node/174
    return np.sum(np.log(stats.chi2.pdf((nPred-1.)*np.power(vObs,2.)/np.power(vPred,2.),nPred-1)))

def logPosterior(params,annuli,rin,rout,nObs,sObs):
    nmean,k,smax,m = params
    nPred,sPred = getProfiles(annuli,rin,rout,1000*nmean,k,smax,m,0)
    nPred/=nmean
    return logPriorK(k)+logPriorM(m)+logPriorSigma0(smax)+logPriorNmean(nmean)+\
           logLikelihoodNu(nPred,nObs)+logLikelihoodSigma(sObs,sPred,nPred)

In [58]:
ntot=1000
bla1,bla2 = getProfiles(annuli,1,1000,1000*ntot,3,1000,0.2,0)
bla1/=ntot

print bla1
print bla2

[119  90 135 112 105  92 116 108 119]
[541.9501154398267, 550.0703909727582, 547.0231297281315, 527.7080407166558, 508.4134354640366, 483.9905690018608, 455.950730561485, 411.2504668835222, 324.79548905832775]


In [61]:
nwalkers = 10
ndim = 4

#initial = np.tile(np.array([np.random.normal(1000,100),np.random.normal(3,0.3),np.random.normal(1000,100),np.random.normal(0.2,0.02)]),10).reshape((nwalkers,ndim))
initial = np.array([np.random.normal(1000,100),np.random.normal(3,0.3),\
                    np.random.normal(1000,100),np.random.normal(0.2,0.02)])

for i in range(nwalkers-1):
    initial=np.vstack((initial,np.array([np.random.normal(1000,100),np.random.normal(3,0.3),\
                    np.random.normal(1000,100),np.random.normal(0.2,0.02)])))

#for i in range(nwalkers):
#    for j in range(ndim):
#        initial[i,j]+=np.random.normal()


In [62]:
rin = 1
rout = 1000
sampler = mc.EnsembleSampler(nwalkers, ndim, logPosterior, args=[annuli,rin,rout,nObs,sObs])

#p0 = np.array([1000,3,1000,0.2])reshape((nwalkers,ndim))

pos, prob, state = sampler.run_mcmc(initial, 1)


emcee: Exception while calling your likelihood function:
  params: [  1.01502153e+03   3.24764095e+00   1.29531098e+03   1.87758408e-01]
  args: [array([   0.        ,    0.79312162,    0.99200463,    1.19653973,
          1.44488527,    1.78940227,    2.26611172,    3.40412606,
          6.47930261,  488.39105307]), 1, 1000, array([112, 111, 112, 111, 112, 111, 112, 111, 112]), [540.6459164824956, 535.2118446805213, 479.94459127400813, 478.113651156466, 505.3241440991127, 495.1272316797748, 450.3661464577093, 348.32470090040425, 383.5558324031583]]

/home/ondrej/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
Traceback (most recent call last):
  File "/home/ondrej/anaconda/lib/python2.7/site-packages/emcee/ensemble.py", line 505, in __call__
    return self.f(x, *self.args, **self.kwargs)



  kwargs: {}
  exception:


  File "<ipython-input-57-2b76d0699331>", line 43, in logPosterior
    nPred,sPred = getProfiles(annuli,rin,rout,1000*nmean,k,smax,m,0)
  File "<ipython-input-55-34a34fb5b8b2>", line 16, in getProfiles
    mock = createMockCluster(rin,rout,nmean,k,smax,m,vmean)
  File "<ipython-input-53-f37c4539df5f>", line 13, in createMockCluster
    vobs = np.array([np.random.normal(loc=vmean,scale=(1/np.sqrt(3))*smax/(i**m)) for i in rho])
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
sampler.run_mcmc(pos, 1000)